# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys2 import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
cities_df = pd.read_csv('../WeatherPy/Output/cities_df.csv')
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chokurdakh,70.6333,147.9167,39.00,71,18,7.65,RU,1627850165
1,Krasnyy Yar,46.5331,48.3456,76.93,55,17,5.12,RU,1627850166
2,Inuvik,68.3499,-133.7218,85.06,29,40,10.36,CA,1627850166
3,Vaini,-21.2000,-175.2000,71.76,83,20,21.85,TO,1627849936
4,Rome,43.2128,-75.4557,66.06,93,75,0.00,US,1627850043


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key)

In [16]:
coordinates = cities_df[["Lat", "Lng"]].astype(float)
humidity_heat = cities_df['Humidity'].astype(float)

 # Create a poverty Heatmap layer
fig = gmaps.figure(center=(20, 5), zoom_level=2)

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity_heat, 
                                 dissipating=False, max_intensity=200,
                                 point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
perfect_weather = cities_df.loc[(cities_df["Max Temp"] < 80) \
                                & (cities_df["Max Temp"] > 70) \
                                & (cities_df["Wind Speed"] < 10) \
                                & (cities_df["Cloudiness"] == 0)]
perfect_weather.dropna()
perfect_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,Koungou,-12.7336,45.2042,73.11,73,0,3.44,YT,1627850203
117,Barcelona,41.3888,2.1590,74.97,64,0,5.75,ES,1627850325
273,Asău,46.4333,26.4000,76.12,90,0,5.08,RO,1627850307
289,Formosa,-26.1775,-58.1781,73.38,35,0,8.05,AR,1627850328
311,Arrondissement d'Ajaccio,42.0000,8.9167,73.45,68,0,3.44,FR,1627850619


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
hotel_df= perfect_weather.copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
73,Koungou,-12.7336,45.2042,73.11,73,0,3.44,YT,1627850203,
117,Barcelona,41.3888,2.1590,74.97,64,0,5.75,ES,1627850325,
273,Asău,46.4333,26.4000,76.12,90,0,5.08,RO,1627850307,
289,Formosa,-26.1775,-58.1781,73.38,35,0,8.05,AR,1627850328,
311,Arrondissement d'Ajaccio,42.0000,8.9167,73.45,68,0,3.44,FR,1627850619,


In [47]:
params = {"type" : "hotel",
          "keyword" : "nearest hotel",
          "radius" : 5000,
          "key" : g_key}


In [48]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_request = requests.get(base_url, params=params)
    hotel_name = hotel_request.json()
    
    
    try:
        hotel_df.loc[index, "Hotel"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):

        print("Error 404: Not found")
    

In [49]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel
73,Koungou,-12.7336,45.2042,73.11,73,0,3.44,YT,1627850203,,"Résidence Beauregard, Majikavo"
117,Barcelona,41.3888,2.1590,74.97,64,0,5.75,ES,1627850325,,W Barcelona
273,Asău,46.4333,26.4000,76.12,90,0,5.08,RO,1627850307,,Pensiunea Sophia
289,Formosa,-26.1775,-58.1781,73.38,35,0,8.05,AR,1627850328,,Howard Johnson by Wyndham Formosa Casino
311,Arrondissement d'Ajaccio,42.0000,8.9167,73.45,68,0,3.44,FR,1627850619,,Sole E Monte


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
